## **Coleta de dados do Datalake da Basedosdados**
***
##### **Objetivo**: coletar dados sobre série histórica da inflação e venda de gasolina pelos distribuidores, armazenados no data lake público da base dos dados
##### **Resultado**: utilizando-se da biblioteca de autoria da organização base dos dados, utilizei a função que direciona o python, por meio da minha conta no google BigQuery, a executar uma query no data lake da própria organização; por fim, armazenei o resultado de ambas as consultas no banco de dados do projeto ('dados.db').
***

In [ ]:
# Bibliotecas utilizadas
import pandas  as pd
import sqlite3 as sql3

from basedosdados import read_sql
from glob         import glob

In [ ]:
# Criando / Fazendo conexão com banco de dados do projeto
conn = sql3.connect( '../analise_dados/dados.db' )

### Racional do código

#### 1 Criei uma função que ler arquivos

In [ ]:
# Criando função para ler os arquivos '.sql'
def ler_query( nome_query ):
    
    # Abrindo arquivo de interesse
    arq = open( nome_query, 'r')
    
    # Lendo conteúdo do arquivo e amazenando na variável
    query = arq.read()
    
    # Fechando arquivo
    arq.close()

    # Retornando o conteúdo do arquivo
    return query

#### 2 Executando query

##### 2.1 armazenei em uma lista o nome dos arquivos '.sql' que devem ser lidos pela função

In [ ]:
# Coletando e armazenando em uma lista os nomes de arquivos com extensão '.sql' a serem
# lidos e executados no datalake
querys = glob( '*.sql' )

##### 2.2 Antes de usar a função para consultar os dados, precisei registrar em uma lista que é necessário a confirmação de identidade (login) na conta do BigQuery apenas na primeira query a ser executada

In [ ]:
# Registrando que só há necessidade de autenticação na primeira consulta do BigQuery;
# Para isso, itero a lista com a quantidade de querys que serão executadas no data lake e
# registro apenas na primeira o comando 'True', autorizando assim a atenticação.
confirmacao = []
for i in range( len( querys ) ):
    if i == 0: confirmacao.append( True  )
    else     : confirmacao.append( False )

##### 2.3 Laço for que itera a lista com nome dos arquivos '.sql'; nele, o arquivo é lido pela função criada e, quando tiver o resultado da consulta, armazenará ele no banco de dados do projeto

In [ ]:
# Executando querys e armazenando resultados em arquivos '.csv' na pasta indicada
for i, item in enumerate( querys ):
    (
        # Consultando BigQuery com query dos arquivos '.sql'
        read_sql(
            ler_query( querys[i] ),
            billing_project_id = 'portfolio-367913',
            reauth = confirmacao[i]
        )
        
        # Armazenando dados da consulta no banco de dados do projeto
        .to_sql(
            f'{item[6:-4]}',
            con       = conn,
            if_exists = 'replace',
            index     = False
        )
    )